In [1]:
from reportlab.lib.pagesizes import letter
from reportlab.platypus import SimpleDocTemplate, Table, TableStyle
from reportlab.lib import colors
import pandas as pd
from datetime import datetime
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
import tabula
import time


In [67]:
from time import sleep
import os
data_range =0
if 'KIBOR.csv' in os.listdir(r'C:\Users\INSHAAL'):
    data = pd.read_csv("KIBOR.csv")
    data['DATE'] = pd.to_datetime(data['DATE'], format='%m/%d/%Y')
    rawdate = data["DATE"].max()
    year = rawdate.year
    month = rawdate.month
    filtered_df = data[(data['DATE'].dt.month == month) & (data['DATE'].dt.year == year)]
    data = data[~data.index.isin(filtered_df.index)]
    data['DATE'] = data['DATE'].dt.strftime('%m/%d/%Y')
    data_range = month -1
    data.to_csv("KIBOR.csv", index = False)
else:
    data_range = 0

driver = webdriver.Chrome()
finaldf = pd.DataFrame()

# Define the URL of the website
url = 'https://www.sbp.org.pk/ecodata/kibor_index.asp'

# Open the website in the browser
driver.get(url)
list_of_months = ['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
for i in range(2023,2024):
    select_year_path =  driver.find_element_by_xpath ('//*[@id="year"]')
    select_year_path.click()
    dropdown_year = Select(select_year_path)
    dropdown_year.select_by_visible_text(str(i))
    sleep(2)
    for j in range(data_range, 12):
        try:
            select_month_path = driver.find_element_by_xpath('//*[@id="month"]')
            select_month_path.click()
            dropdown_month = Select(select_month_path)
            dropdown_month.select_by_visible_text(list_of_months[j])
        except:
            continue
        sleep(2)
        links = driver.find_elements(By.TAG_NAME, 'a')
        for link in links:
            href = link.get_attribute('href')
            if href and href.endswith('.pdf'):
                # Use tabula to extract tables from the PDF
                tables = tabula.read_pdf(href, pages='all')
                table_df = tables[0]
                date = str(href)
                date = date[52:59] + '20' + date[59:61]
                parsed_date = datetime.strptime(date, "%d-%b-%Y")
                formatted_date = parsed_date.strftime("%d-%m-%Y")
                table_df['Date'] = formatted_date
                table_df['Date'] = pd.to_datetime(table_df['Date'], format='%d-%m-%Y', dayfirst=True)
                date_column = table_df.pop('Date')
                table_df.insert(0, 'Date', date_column)
                table_df.to_csv('KIBOR.csv',mode='a', index = False, header = False)
                
driver.quit()